In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [2]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences



In [3]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [4]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [8]:
def target_base_CT_plus(seq):
    match = re.findall('(?=(\w{20}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [9]:
def target_base_AG_plus(seq):
    match = re.findall('(?=(CC\w{20}))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][-3:]
        if target =='TGG':#kここはCCA?
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [10]:
def target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [11]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [12]:
def exon_data(name,sequence_data):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    for s in range(len(start_list)-1):
        start_num = int(start_list[s])- set_num
        
        end_num= int(end_list[s]) - set_num
    
        
        exon_seq =sequence_data[start_num:end_num]
        exon_seq_list.append(exon_seq)
        
    
    
    return ''.join(exon_seq_list)

In [13]:
def where_start_codon(name):
    data = gene_df[gene_df['name']==name]
    cdsStart =int(data['cdsStart'].to_list()[0])
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    for s in range(len(start)-1):
        if (int(start[s]) <= cdsStart<=int(end[s])):
             exon_num =s
            
    return exon_num


In [14]:
def start_codon(name,exon_data,exon_num):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    CDs =''
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    cds_list=[]
    if exon_num == 0:
            cds_list.append(exon_data[set_num:])
            
    
    
    else:     
        start_num = int(data['cdsStart'].to_list()[0])- int(start[exon_num])

        for s in range(exon_num):
            start_num += int(end[s])-int(start[s])
        cds_list.append(exon_data[start_num:])
        
    return ''.join(cds_list)

In [15]:
def end_codon(cds_seq):
    matches = re.finditer('(?=(CAA)|(?=(CAG))|(?=(CGA))|(?=(TGG)))', cds_seq)
    result=[]
    for match in matches:
        start_pos = match.start() 
        
        if (start_pos % 3) == 0:
            result.append(start_pos)
    return result

In [ ]:
#こいつらに要修正

In [16]:
def add_num_list(name):#修正が必要
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_list=[]
    num = 0
    for s in range(len(start)-1):
        element_list=[]
        element_list.append(num)
        
        num += int(end[s])-int(start[s])
        
        element_list.append(num)
        add_list.append(element_list)
        
    return add_list

In [259]:
def add_num_correct(name,number_list, interval_list,exon_num):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    
    number_list = np.array(number_list)
    start_num=int(interval_list[exon_num][0])
    number_list = number_list + (set_num-int(start[exon_num])+start_num)
    number_list = number_list.tolist()#ここでエクソン上で何番目かになってる（cdsからエクソン上に）
    print(number_list)
    
    
    indices = []
    for number in number_list:
        for index, interval in enumerate(interval_list):
           # print(index)
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
    n =0        
    for i in indices:
        
        if i == 0:
            add_result.append(set_num)
            n +=1
        else:
            add_result.append(int(number_list[n])-int(interval_list[i][0]))
            n += 1
        
    print(add_result)
    return add_result

In [260]:
def add_num(name,number_list, interval_list,exon_num):
    print(interval_list)
    print(number_list)
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    
    number_list = np.array(number_list)
    start_num=int(interval_list[exon_num][0])
    number_list = number_list + (set_num-int(start[exon_num])+start_num)
    number_list = number_list.tolist()#ここでエクソン上で何番目かになってる（cdsからエクソン上に）
    print(number_list)
    
    
    indices = []
    for number in number_list:
        for index, interval in enumerate(interval_list):
           # print(index)
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
            
    for i in indices:
        if i == 0:
            add_result.append(set_num)
        else:
            add_result.append(int(start[i]))
            
    print(add_result)
    return add_result

In [261]:
gene_df[gene_df['name']=="NM_001301868"]

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
5050,Ikzf1,NM_001301868,chr11,+,11635924,11722926,11650218,11719581,8,"11635924,11650204,11657784,11698310,11704006,1...","11636468,11650258,11657904,11698571,11704174,1..."


In [262]:
a = exon_data("NM_001301868",base_data['NM_001301868::chr11:11635924-11722926'])

In [263]:
base_data['NM_001301868::chr11:11635924-11722926'][11723161-11635924:]

''

In [264]:
a[5397:]

'CAATCCTTTCTAATACATTTTTTC'

In [265]:
start_codon("NM_001301868",a,1)

'ATGGATGTCGATGAGGGTCAAGACATGTCCCAAGTTTCAGGAAAGGAGAGCCCCCCAGTCAGTGACACTCCAGATGAAGGGGATGAGCCCATGCCTGTCCCTGAGGACCTGTCCACTACCTCTGGAGCACAGCAGAACTCCAAGAGTGATCGAGGCATGGCCAGTAATGTTAAAGTAGAGACTCAGAGTGATGAAGAGAATGGGCGTGCCTGTGAAATGAATGGGGAAGAATGTGCAGAGGATTTACGAATGCTTGATGCCTCGGGAGAGAAAATGAATGGCTCCCACAGGGACCAAGGCAGCTCGGCTTTGTCAGGAGTTGGAGGCATTCGACTTCCTAACGGAAAACTAAAGTGTGATATCTGTGGGATCGTTTGCATCGGGCCCAATGTGCTCATGGTTCACAAAAGAAGTCATACTGGTGAACGGCCTTTCCAGTGCAACCAGTGTGGGGCCTCCTTTACCCAGAAAGGCAACCTCCTGCGGCACATCAAGCTGCACTCGGGTGAGAAGCCCTTCAAATGCCATCTTTGCAACTATGCCTGCCGCCGGAGGGACGCCCTCACCGGCCACCTGAGGACGCACTCCGTTGGTAAGCCTCACAAATGTGGATATTGTGGCCGGAGCTATAAACAGCGAAGCTCTTTAGAGGAGCATAAAGAGCGATGCCACAACTACTTGGAAAGCATGGGCCTTCCGGGCATGTACCCAGTCATTAAGGAAGAAACTAACCACAACGAGATGGCAGAAGACCTGTGCAAGATAGGAGCAGAGAGGTCCCTTGTCCTGGACAGGCTGGCAAGCAATGTCGCCAAACGAGACAAGTGCCTGTCAGACATGCCCTATGACAGTGCCAACTATGAGAAGGAGGATATGATGACATCCCACGTGATGGACCAGGCCATCAACAATGCCATCAACTACCTGGGGGCTGAGTCCCTGCGCCCATTGGTGCAGACACCCCCCGGTAGCTCCGAGGTGGTGCCAGTCATCAGCTCC

In [276]:
CT_target_base_strand_plus("NM_001301868")

[[0, 544], [544, 598], [598, 718], [718, 979], [979, 1147], [1147, 1270], [1270, 1375], [1375, 5421]]
[18, 30, 129, 132, 150, 183, 246, 294, 330, 435, 444, 465, 633, 636, 663, 816, 897, 954, 1005, 1056, 1113, 1140, 1176, 1251, 1293, 1329, 1455, 1521, 1542, 1572, 1584, 1611, 1635, 1734, 1908, 1917, 1950, 1962, 1971, 2004, 2013, 2022, 2076, 2079, 2091, 2190, 2250, 2283, 2295, 2355, 2394, 2433, 2454, 2478, 2589, 2592, 2664, 2694, 2715, 2745, 2775, 2799, 2820, 2829, 2859, 2907, 2937, 2940, 3027, 3210, 3246, 3264, 3291, 3297, 3360, 3459, 3501, 3645, 3672, 3720, 3729, 3735, 3756, 3759, 3852, 3864, 3885, 3978, 4125, 4134, 4194, 4197, 4200, 4245, 4305, 4404, 4443, 4446, 4629, 4638, 4647, 4665, 4746, 4779, 4827, 4839]
[576, 588, 687, 690, 708, 741, 804, 852, 888, 993, 1002, 1023, 1191, 1194, 1221, 1374, 1455, 1512, 1563, 1614, 1671, 1698, 1734, 1809, 1851, 1887, 2013, 2079, 2100, 2130, 2142, 2169, 2193, 2292, 2466, 2475, 2508, 2520, 2529, 2562, 2571, 2580, 2634, 2637, 2649, 2748, 2808, 2841, 28

array([11650248])

In [273]:
def CT_target_base_strand_plus(name,):
    name_df=gene_df[gene_df['name']==name].reset_index()
    chrom = str(name_df.loc[0,'chrom'])
    txStart = str(name_df.loc[0,'txStart'])
    txEnd = str(name_df.loc[0,'txEnd'])
    
    query = f'{name}::{chrom}:{txStart}-{txEnd}'
    seq = base_data[query]
    available_base_num =target_gene_start(name,target_start(target_base_CT_plus(seq),seq))#配列的に狙えるほう
    
    
    
    start_exon_num= where_start_codon(name)
    end_codon_num =end_codon(start_codon(name,exon_data(name,seq),start_exon_num))   
    add_list =add_num(name,end_codon_num,add_num_list(name),start_exon_num)
 
    end_codon_num=add_num_correct(name,end_codon_num,add_num_list(name),start_exon_num)
    
    available_base_num =np.array(available_base_num)
    end_codon_num = np.array(end_codon_num)
    add_list = np.array(add_list)
 
    
    num_able = available_base_num
    num_codon = end_codon_num+add_list
    print(num_codon)
    num_codon = [x for x in num_codon if x <= int(name_df.loc[0,'cdsEnd'])]
    result = np.intersect1d(num_able,num_codon)

    return result

In [274]:
def AG_target_base_strand_plus(name,):
    name_df=gene_df[gene_df['name']==name].reset_index()
    chrom = str(name_df.loc[0,'chrom'])
    txStart = str(name_df.loc[0,'txStart'])
    txEnd = str(name_df.loc[0,'txEnd'])
    
    query = f'{name}::{chrom}:{txStart}-{txEnd}'
    seq = base_data[query]
    available_base_num =target_gene_start(name,target_start(target_base_AG_plus(seq),seq))#配列的に狙えるほう

    
    start_exon_num= where_start_codon(name)
    end_codon_num =end_codon(start_codon(name,exon_data(name,seq),start_exon_num))   
    add_list =add_num(name,end_codon_num,add_num_list(name),start_exon_num)
    
  
    
    available_base_num =np.array(available_base_num)
    end_codon_num = np.array(end_codon_num)
    add_list = np.array(add_list)

    num_able = available_base_num+19#GGXXXなので19たした
    num_codon = end_codon_num+add_list
    
    num_codon = [x for x in num_codon if x <= int(name_df.loc[0,'cdsEnd'])]
 
    result = np.intersect1d(num_able,num_codon)
  
    return result

In [275]:
CT_target_base_strand_plus("NM_001385633")

[[0, 1307], [1307, 2415], [2415, 2610], [2610, 2670], [2670, 5553]]
[21, 39, 138, 525, 561, 576, 744, 804, 807, 816, 819, 831, 975, 1074, 1485, 1488, 1542, 1659, 1704, 1950, 2049, 2118, 2169, 2427, 2439, 2484, 2493, 2499, 2502, 2559, 2655, 2721, 2733, 2736, 2760, 2826, 2841, 2970, 3000, 3054, 3060, 3093, 3234, 3246, 3363, 3408, 3423, 3444, 3465, 3519, 3525, 3543, 3567, 3603, 3612, 3648, 3708, 3855, 3903, 3915, 3921, 4005, 4014, 4179, 4251, 4275, 4284, 4296, 4380, 4581, 4695, 4698, 4797, 4821, 4836, 4878, 4896, 4902, 4911, 4935, 5013]
[455, 473, 572, 959, 995, 1010, 1178, 1238, 1241, 1250, 1253, 1265, 1409, 1508, 1919, 1922, 1976, 2093, 2138, 2384, 2483, 2552, 2603, 2861, 2873, 2918, 2927, 2933, 2936, 2993, 3089, 3155, 3167, 3170, 3194, 3260, 3275, 3404, 3434, 3488, 3494, 3527, 3668, 3680, 3797, 3842, 3857, 3878, 3899, 3953, 3959, 3977, 4001, 4037, 4046, 4082, 4142, 4289, 4337, 4349, 4355, 4439, 4448, 4613, 4685, 4709, 4718, 4730, 4814, 5015, 5129, 5132, 5231, 5255, 5270, 5312, 5330, 53

array([44244686])

In [277]:
CT_target_base_strand_plus("NM_001301868")

[[0, 544], [544, 598], [598, 718], [718, 979], [979, 1147], [1147, 1270], [1270, 1375], [1375, 5421]]
[18, 30, 129, 132, 150, 183, 246, 294, 330, 435, 444, 465, 633, 636, 663, 816, 897, 954, 1005, 1056, 1113, 1140, 1176, 1251, 1293, 1329, 1455, 1521, 1542, 1572, 1584, 1611, 1635, 1734, 1908, 1917, 1950, 1962, 1971, 2004, 2013, 2022, 2076, 2079, 2091, 2190, 2250, 2283, 2295, 2355, 2394, 2433, 2454, 2478, 2589, 2592, 2664, 2694, 2715, 2745, 2775, 2799, 2820, 2829, 2859, 2907, 2937, 2940, 3027, 3210, 3246, 3264, 3291, 3297, 3360, 3459, 3501, 3645, 3672, 3720, 3729, 3735, 3756, 3759, 3852, 3864, 3885, 3978, 4125, 4134, 4194, 4197, 4200, 4245, 4305, 4404, 4443, 4446, 4629, 4638, 4647, 4665, 4746, 4779, 4827, 4839]
[576, 588, 687, 690, 708, 741, 804, 852, 888, 993, 1002, 1023, 1191, 1194, 1221, 1374, 1455, 1512, 1563, 1614, 1671, 1698, 1734, 1809, 1851, 1887, 2013, 2079, 2100, 2130, 2142, 2169, 2193, 2292, 2466, 2475, 2508, 2520, 2529, 2562, 2571, 2580, 2634, 2637, 2649, 2748, 2808, 2841, 28

array([11650248])

In [278]:
AG_target_base_strand_plus("NM_001385633")

[[0, 1307], [1307, 2415], [2415, 2610], [2610, 2670], [2670, 5553]]
[21, 39, 138, 525, 561, 576, 744, 804, 807, 816, 819, 831, 975, 1074, 1485, 1488, 1542, 1659, 1704, 1950, 2049, 2118, 2169, 2427, 2439, 2484, 2493, 2499, 2502, 2559, 2655, 2721, 2733, 2736, 2760, 2826, 2841, 2970, 3000, 3054, 3060, 3093, 3234, 3246, 3363, 3408, 3423, 3444, 3465, 3519, 3525, 3543, 3567, 3603, 3612, 3648, 3708, 3855, 3903, 3915, 3921, 4005, 4014, 4179, 4251, 4275, 4284, 4296, 4380, 4581, 4695, 4698, 4797, 4821, 4836, 4878, 4896, 4902, 4911, 4935, 5013]
[455, 473, 572, 959, 995, 1010, 1178, 1238, 1241, 1250, 1253, 1265, 1409, 1508, 1919, 1922, 1976, 2093, 2138, 2384, 2483, 2552, 2603, 2861, 2873, 2918, 2927, 2933, 2936, 2993, 3089, 3155, 3167, 3170, 3194, 3260, 3275, 3404, 3434, 3488, 3494, 3527, 3668, 3680, 3797, 3842, 3857, 3878, 3899, 3953, 3959, 3977, 4001, 4037, 4046, 4082, 4142, 4289, 4337, 4349, 4355, 4439, 4448, 4613, 4685, 4709, 4718, 4730, 4814, 5015, 5129, 5132, 5231, 5255, 5270, 5312, 5330, 53

array([], dtype=int32)

In [279]:
AG_target_base_strand_plus("NM_001301868")

[[0, 544], [544, 598], [598, 718], [718, 979], [979, 1147], [1147, 1270], [1270, 1375], [1375, 5421]]
[18, 30, 129, 132, 150, 183, 246, 294, 330, 435, 444, 465, 633, 636, 663, 816, 897, 954, 1005, 1056, 1113, 1140, 1176, 1251, 1293, 1329, 1455, 1521, 1542, 1572, 1584, 1611, 1635, 1734, 1908, 1917, 1950, 1962, 1971, 2004, 2013, 2022, 2076, 2079, 2091, 2190, 2250, 2283, 2295, 2355, 2394, 2433, 2454, 2478, 2589, 2592, 2664, 2694, 2715, 2745, 2775, 2799, 2820, 2829, 2859, 2907, 2937, 2940, 3027, 3210, 3246, 3264, 3291, 3297, 3360, 3459, 3501, 3645, 3672, 3720, 3729, 3735, 3756, 3759, 3852, 3864, 3885, 3978, 4125, 4134, 4194, 4197, 4200, 4245, 4305, 4404, 4443, 4446, 4629, 4638, 4647, 4665, 4746, 4779, 4827, 4839]
[576, 588, 687, 690, 708, 741, 804, 852, 888, 993, 1002, 1023, 1191, 1194, 1221, 1374, 1455, 1512, 1563, 1614, 1671, 1698, 1734, 1809, 1851, 1887, 2013, 2079, 2100, 2130, 2142, 2169, 2193, 2292, 2466, 2475, 2508, 2520, 2529, 2562, 2571, 2580, 2634, 2637, 2649, 2748, 2808, 2841, 28

array([], dtype=int32)

In [280]:
gene_df[gene_df['geneName']=='Sulf1']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
77,Sulf1,NM_001198565,chr1,+,12762653,12930596,12856750,12929066,22,"12762653,12798817,12856302,12856691,12866989,1...","12762908,12798912,12856375,12856922,12867229,1..."
78,Sulf1,NM_001198566,chr1,+,12762653,12930596,12856750,12929066,21,"12762653,12856302,12856691,12866989,12875403,1...","12762908,12856375,12856922,12867229,12875555,1..."
79,Sulf1,NM_172294,chr1,+,12788768,12930596,12856750,12929066,22,"12788768,12798817,12856302,12856691,12866989,1...","12789007,12798912,12856375,12856922,12867229,1..."


In [281]:
CT_target_base_strand_plus("NM_172294")

[[0, 239], [239, 334], [334, 407], [407, 638], [638, 878], [878, 1030], [1030, 1200], [1200, 1351], [1351, 1527], [1527, 1656], [1656, 1713], [1713, 1843], [1843, 2060], [2060, 2313], [2313, 2410], [2410, 2505], [2505, 2571], [2571, 2747], [2747, 2890], [2890, 3014], [3014, 3048], [3048, 4609]]
[15, 45, 81, 90, 102, 105, 111, 156, 180, 213, 348, 351, 465, 468, 474, 696, 726, 774, 783, 834, 843, 849, 966, 1068, 1218, 1287, 1290, 1335, 1338, 1350, 1365, 1374, 1380, 1383, 1413, 1458, 1548, 1560, 1566, 1623, 1692, 1740, 1758, 1896, 1911, 1956, 2034, 2067, 2070, 2118, 2139, 2193, 2256, 2259, 2274, 2325, 2421, 2466, 2478, 2493, 2502, 2514, 2586, 2592, 2601, 2619, 2637, 2673, 2679, 2688, 2745, 2766, 2784, 2793, 2814, 2901, 2931, 2955, 3096, 3099, 3102, 3105, 3138, 3153, 3240, 3264, 3297, 3321, 3363, 3369, 3453, 3456, 3459, 3549, 3564, 3591, 3639, 3699, 3708, 3825, 3840, 3867, 3885, 3954, 4020, 4077, 4122, 4134]
[481, 511, 547, 556, 568, 571, 577, 622, 646, 679, 814, 817, 931, 934, 940, 1162, 

array([12908728, 12910101, 12918596])

In [282]:
CT_target_base_strand_plus("NM_172294")

[[0, 239], [239, 334], [334, 407], [407, 638], [638, 878], [878, 1030], [1030, 1200], [1200, 1351], [1351, 1527], [1527, 1656], [1656, 1713], [1713, 1843], [1843, 2060], [2060, 2313], [2313, 2410], [2410, 2505], [2505, 2571], [2571, 2747], [2747, 2890], [2890, 3014], [3014, 3048], [3048, 4609]]
[15, 45, 81, 90, 102, 105, 111, 156, 180, 213, 348, 351, 465, 468, 474, 696, 726, 774, 783, 834, 843, 849, 966, 1068, 1218, 1287, 1290, 1335, 1338, 1350, 1365, 1374, 1380, 1383, 1413, 1458, 1548, 1560, 1566, 1623, 1692, 1740, 1758, 1896, 1911, 1956, 2034, 2067, 2070, 2118, 2139, 2193, 2256, 2259, 2274, 2325, 2421, 2466, 2478, 2493, 2502, 2514, 2586, 2592, 2601, 2619, 2637, 2673, 2679, 2688, 2745, 2766, 2784, 2793, 2814, 2901, 2931, 2955, 3096, 3099, 3102, 3105, 3138, 3153, 3240, 3264, 3297, 3321, 3363, 3369, 3453, 3456, 3459, 3549, 3564, 3591, 3639, 3699, 3708, 3825, 3840, 3867, 3885, 3954, 4020, 4077, 4122, 4134]
[481, 511, 547, 556, 568, 571, 577, 622, 646, 679, 814, 817, 931, 934, 940, 1162, 

array([12908728, 12910101, 12918596])